In [ ]:
"""
peft、transformers的deepseek-distll-8B的lora微调

requirements:
transformers
datasets
peft
bitsandbytes
"""

In [1]:
# 1.加载一个huggingface的deepseek模型
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "models/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

/root/autodl-tmp/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2.找个简单的数据集微调
from datasets import load_dataset
# 取100条
dataset = load_dataset("parquet", data_files="datasets/train-00000-of-00001.parquet", split="train[:100]")
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [ ]:
# 3.tokenize函数
def tokenize_data(samples):
    texts = [f"{prompt}\n{completion}" for prompt, completion in zip(samples["prompt"], samples["completion"])]
    inputs = tokenizer(texts, truncation=True, padding=True, max_length=512)
    inputs["labels"] = inputs["input_ids"].copy() # 这样的方式可能并不好

    return inputs

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

Map: 100%|██████████| 10/10 [00:00<00:00, 507.22 examples/s]


In [ ]:
# 量化设置并加载模型
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    quantization_config=quantization_config, 
    device_map="auto"
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
# 配置lora
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.001,
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(
    model=model,
    peft_config=lora_config,
)

In [14]:
# train params
from transformers import TrainingArguments
train_args = TrainingArguments(
    output_dir="./deepseek-disill-1.5b-casual_finetune_0.1k",
    eval_strategy="steps",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    fp16=True,
    logging_steps=10,
    save_steps=50,
    num_train_epochs=20,
    do_eval=True,
    do_train=True,
    logging_dir="./logs",
    run_name="deepseek-disill-1.5b-casual_finetune_0.1k"
)

In [15]:
# trainer
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
trainer.train()

Step,Training Loss,Validation Loss
10,2.163500,2.283011
20,2.193400,2.226271
30,2.088000,2.176583
40,1.970200,2.125116
50,2.131700,2.054769
60,2.071400,1.993876
70,1.893100,1.911018
80,1.851800,1.839321
90,1.726300,1.776606
100,1.795900,1.718472


TrainOutput(global_step=440, training_loss=1.5642448165199974, metrics={'train_runtime': 399.7509, 'train_samples_per_second': 4.503, 'train_steps_per_second': 1.101, 'total_flos': 8171987596738560.0, 'train_loss': 1.5642448165199974, 'epoch': 19.133333333333333})

In [ ]:
# lora模型保存
save_path = "deepseek-r1-1.5b-1k-lora"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
# 合并原始模型和lora模型并保存
lora_and_model_merge_save_path = "deepseek-r1-1.5b-1k-merge"
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(model_path)
model = PeftModel(base_model, save_path)
model = model.merge_and_unload()

model.save_pretrained(lora_and_model_merge_save_path)
tokenizer.save_pretrained(lora_and_model_merge_save_path)

In [ ]:
# 构建pipeline推理
from transformers import AutoModelForCausalLM, AutoTokenizer
lora_and_model_merge_save_path = "deepseek-r1-1.5b-1k-merge"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "tell me a joke about sun."
generated_texts = pipe(prompt, max_length=512, num_return_sequences=1)
print("开始回答----------", generated_texts[0]["generated_text"])
